# FitMut2.0
In this notebook we walk through an example use case, generating data from simulation and then inferring fitness and establishment times. Though we expect most users will want to use our code to infer mutational parameters on their own experimental datasets, we include instructions on how to generate simulated data for the sake of completeness.

First we install Python packages that required

In [ ]:
!pip install -r ../requirements.txt

Then we generate the input files required to run a simulation. These include COMPLETE THIS
specification of $\mu(s)$ etc.

In [2]:
# input for fitmutsimu_run_v1.py
import numpy as np
import itertools
import csv

dir_simulation = './'

delta_t = 8
t_seq = np.arange(0, delta_t*15, delta_t)

tmp_1 = {'0':t_seq, 
         '1':10*np.ones(np.shape(t_seq)), 
         '2':20*np.ones(np.shape(t_seq)), 
         '3':50*np.ones(np.shape(t_seq)), 
         '4':100*np.ones(np.shape(t_seq))}


tmp = list(itertools.zip_longest(*list(tmp_1.values())))
with open(dir_simulation + 'simu_input_time_points.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(tmp)
        
        
step_size = 0.001
np.random.seed(5)
s = np.random.exponential(0.045,100000) 
bins_edge = np.arange(0, 0.145, step_size)
        
freq_bin = np.histogram(s, bins=bins_edge, density=True)[0] * step_size
input_test = {'0':[1e-5], '1':bins_edge, '2':freq_bin}
tmp = list(itertools.zip_longest(*list(input_test.values())))
with open(dir_simulation + 'simu_input_mutation_fitness.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(tmp)


Then we run the commend (fitmutsimu_run_v1.py) to generate simulated data.

In [3]:
import os

dir_simulation = './'
dir_code = '../main_code/'
        
lineages_num = int(1e4)

os.system('python ' + dir_code + 'fitmutsimu_run_v1.py -l %i -t '%lineages_num
              + dir_simulation + 'simu_input_time_points.csv -s '
              + dir_simulation + 'simu_input_mutation_fitness.csv -o '
              + dir_simulation + 'simu')


0

Next, we generate the input files required to run the inference algorithm.

In [4]:
# input for fitmut2_run_v1.py
import numpy as np
import pandas as pd
import itertools
import csv

dir_simulation = './'
dir_result = './'

csv_input = pd.read_csv(dir_simulation + 'simu_input_time_points.csv', low_memory=False, header=None)
t_seq = np.array(csv_input[0][~pd.isnull(csv_input[0])], dtype=float)
cell_num_average_bottleneck = 100
t_delta = 8

csv_input = pd.read_csv(dir_simulation + 'simu_test_0_EvoSimulation_Read_Number.csv', low_memory=False, header=None)
lineages_num = csv_input.shape[0]
 
cell_depth_seq = cell_num_average_bottleneck*lineages_num*np.ones(t_seq.shape)*t_delta
input_tmp = {'0':t_seq, '1':cell_depth_seq}
tmp = list(itertools.zip_longest(*list(input_tmp.values())))
with open(dir_result + 'fitmut_input_time_points.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(tmp)


Finally we run the commend (fitmut2_run_v1.py) to detect adaptive mutations and infer their fitness effects and establishment time using simulated data.

In [5]:
import os

dir_code = '../main_code/'
dir_simulation = './'
dir_result = './'

os.system('python ' + dir_code + 'fitmut2_run_v1.py -i ' 
          + dir_simulation + 'simu_3_EvoSimulation_Read_Number.csv -t '
          + dir_result + 'fitmut_input_time_points.csv -o ' 
          + dir_result + 'test')


0